In [4]:
# import modules
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import SVD, SlopeOne, NMF, KNNWithMeans
from surprise.accuracy import rmse
from surprise import Dataset, accuracy
from surprise.model_selection import train_test_split
import os
import time
from collections import defaultdict

In [5]:
# import dataset

restaurant_df = pd.read_csv('restaurant.csv')
rate_df = pd.read_csv('user_reviews.csv')

In [6]:
restaurant_df.head(5)

,restaurant_id,restaurant_name,neighborhood,review_count,price,type,average_rating
0,_IFMCrheTi12RqiR7jvJUg,Little Rey,Westside / Home Park,225,2.0,mexican,4.0
1,0qJXSTFBnNoh5lMD3LgE0Q,Bacchanalia,Westside / Home Park,660,4.0,american,4.5
2,0ukhgBeLATOo8b8lDobRGg,Roasters,Westside / Home Park,369,2.0,american,4.0
3,14nIlMhxTXWnJg0Glrr-PQ,Top Spice,Morningside / Lenox Park,471,2.0,asian,4.0
4,1i63faxXI1TQ7pNlLp3IPQ,Cypress Street Pint & Plate,Westside / Home Park,1244,2.0,american,4.0


In [7]:
rate_df.head(5)

,restaurant_name,restaurant_id,user_id,friends,number_reviews,photos,area_AL,elite_user,date,rating,rating_mean,standard_dev
0,Fox Bros. Bar-B-Q,u-4wti774tFcYRLuQrnHEg,__1kMkvHH-kWVeokwZSFXw,115,3,NaN,1,0,3/18/2016,5,4.268636,NaN
1,South City Kitchen Midtown,eG-UO83g_5zDk70FIJbm2w,__48dJJcPvNgqUlEozwtpw,105,318,1219.0,0,0,8/23/2014,4,4.336328,NaN
2,The Vortex Bar And Grill - Midtown,Z2qMwUhnGt_2pA9uQbS7Uw,__48dJJcPvNgqUlEozwtpw,105,318,1219.0,0,0,8/23/2014,4,3.946784,NaN
3,Fat Matt's Rib Shack,ALYQ-uM_uMkKbkXlhWcgbQ,__48dJJcPvNgqUlEozwtpw,105,318,1219.0,0,0,8/23/2014,5,4.178538,NaN
4,Cypress Street Pint & Plate,1i63faxXI1TQ7pNlLp3IPQ,__48dJJcPvNgqUlEozwtpw,105,318,1219.0,0,0,8/23/2014,4,4.028112,NaN


In [8]:
print('restaurant: ', restaurant_df.shape)
print('user_reviews: ', rate_df.shape)

restaurant:  (97, 7)
user_reviews:  (71432, 12)


In [9]:
# generate different size of datasets
rate_df_1 = rate_df[0:2000]
rate_df_2 = rate_df[0:20000]
rate_df_3 = rate_df[0:70000]

In [10]:
print('user_reviews_1: ', rate_df_1.shape)
print('user_reviews_2: ', rate_df_2.shape)
print('user_reviews_3: ', rate_df_3.shape)

user_reviews_1:  (2000, 12)
user_reviews_2:  (20000, 12)
user_reviews_3:  (70000, 12)


In [11]:
# data merge

full_df = rate_df_1.merge(restaurant_df, how='left', left_on=['restaurant_id'], right_on=['restaurant_id'])
full_df.head()

,restaurant_name_x,restaurant_id,user_id,friends,number_reviews,photos,area_AL,elite_user,date,rating,rating_mean,standard_dev,restaurant_name_y,neighborhood,review_count,price,type,average_rating
0,Fox Bros. Bar-B-Q,u-4wti774tFcYRLuQrnHEg,__1kMkvHH-kWVeokwZSFXw,115,3,NaN,1,0,3/18/2016,5,4.268636,NaN,Fox Bros. Bar-B-Q,Poncey - Highland,3690,2.0,meat,4.5
1,South City Kitchen Midtown,eG-UO83g_5zDk70FIJbm2w,__48dJJcPvNgqUlEozwtpw,105,318,1219.0,0,0,8/23/2014,4,4.336328,NaN,South City Kitchen Midtown,Westside / Home Park,2563,2.0,american,4.5
2,The Vortex Bar And Grill - Midtown,Z2qMwUhnGt_2pA9uQbS7Uw,__48dJJcPvNgqUlEozwtpw,105,318,1219.0,0,0,8/23/2014,4,3.946784,NaN,The Vortex Bar And Grill - Midtown,Westside / Home Park,1710,2.0,fastfood,4.0
3,Fat Matt's Rib Shack,ALYQ-uM_uMkKbkXlhWcgbQ,__48dJJcPvNgqUlEozwtpw,105,318,1219.0,0,0,8/23/2014,5,4.178538,NaN,Fat Matt's Rib Shack,Westside / Home Park,2107,2.0,meat,4.0
4,Cypress Street Pint & Plate,1i63faxXI1TQ7pNlLp3IPQ,__48dJJcPvNgqUlEozwtpw,105,318,1219.0,0,0,8/23/2014,4,4.028112,NaN,Cypress Street Pint & Plate,Westside / Home Park,1244,2.0,american,4.0


In [12]:
# data Analysis

num_of_rate_per_user = full_df.groupby(by='user_id')['rating'].count().sort_values(ascending=True)
num_of_rate_per_user.head()

user_id
-0AyZxS5C--WySnbW_Q8yQ    1
_K9sKlA4fVkWI4hyGSpoPA    1
_K6MfV0SE0dx4rXRYuVe5A    1
_K1HBt6LXnyFKk1nltbD6A    1
_Jha8QCICYz41c2G6LKsrg    1
Name: rating, dtype: int64

In [13]:
full_df = full_df[['user_id', 'restaurant_name_x', 'rating']]
full_df.head()

,user_id,restaurant_name_x,rating
0,__1kMkvHH-kWVeokwZSFXw,Fox Bros. Bar-B-Q,5
1,__48dJJcPvNgqUlEozwtpw,South City Kitchen Midtown,4
2,__48dJJcPvNgqUlEozwtpw,The Vortex Bar And Grill - Midtown,4
3,__48dJJcPvNgqUlEozwtpw,Fat Matt's Rib Shack,5
4,__48dJJcPvNgqUlEozwtpw,Cypress Street Pint & Plate,4


In [14]:
print(full_df.shape)

(2000, 3)


In [79]:
#Getting the new dataframe which contains users who has given 2 or more ratings

data=full_df.groupby("user_id").filter(lambda x:x['rating'].count() >=2)
data.head(10)

,user_id,restaurant_name_x,rating
1,__48dJJcPvNgqUlEozwtpw,South City Kitchen Midtown,4
2,__48dJJcPvNgqUlEozwtpw,The Vortex Bar And Grill - Midtown,4
3,__48dJJcPvNgqUlEozwtpw,Fat Matt's Rib Shack,5
4,__48dJJcPvNgqUlEozwtpw,Cypress Street Pint & Plate,4
8,__bMs0nf3_hnhitK91gT4A,South City Kitchen Midtown,4
9,__bMs0nf3_hnhitK91gT4A,Atlanta Breakfast Club,5
10,__bMs0nf3_hnhitK91gT4A,Herban Fix - Vegan Kitchen,2
18,_02XN3yATdWwfMIbsGhMuQ,Slutty Vegan,5
19,_02XN3yATdWwfMIbsGhMuQ,26 Thai Kitchen & Bar,4
20,_02XN3yATdWwfMIbsGhMuQ,Sweet Georgia's Juke Joint,2


In [80]:
print(data.shape)

(911, 3)


In [56]:
# Build collaborative filtering Models!
# Codes referenced "https://www.kaggle.com/malikasif123/amazon-reviews-recommendation-system"
# & "https://www.kaggle.com/podsyp/anime-recommendations-with-surprise"

In [82]:
#Reading the dataset using Surprise package for User Based Collaborative Filtering

reader = Reader(rating_scale=(1, 5))
data_reader_SVD = Dataset.load_from_df(data,reader)

In [83]:
# 5-fold cross validation

%time
algo = SVD(biased=False)
cross_validate(algo, data_reader_SVD, measures=['RMSE'], cv=5, verbose=False)

Wall time: 0 ns


{'test_rmse': array([3.02286858, 3.08915157, 3.00297495, 3.23626693, 3.07087603]),
 'fit_time': (0.07479453086853027,
  0.043883323669433594,
  0.03291201591491699,
  0.03790712356567383,
  0.03786110877990723),
 'test_time': (0.0019960403442382812,
  0.0,
  0.0010347366333007812,
  0.0009503364562988281,
  0.0019981861114501953)}

In [84]:
trainset, testset = train_test_split(data_reader_SVD, test_size=0.25)

In [85]:
# Fitting and evaluating Collobarative filtering
start_time = time.time()
def accuracy_over_n_queries(n = 10):
    for i in range(n):
        predictions = algo.fit(trainset).test(testset)
        accuracy.rmse(predictions)  
    return predictions   
predictions = accuracy_over_n_queries(10)

print(time.time() - start_time)

RMSE: 3.0871
RMSE: 3.0859
RMSE: 3.0863
RMSE: 3.0792
RMSE: 3.0858
RMSE: 3.0867
RMSE: 3.0873
RMSE: 3.0949
RMSE: 3.0763
RMSE: 3.0897
0.36601901054382324


In [86]:
#checking prediction

predictions[0]

Prediction(uid='_hTADMIAWVZW91Fu0ebjcg', iid='Heirloom Market BBQ', r_ui=5.0, est=1, details={'was_impossible': False})

In [89]:
def get_Iu(uid):
    try:
        return len(trainset.ur[trainset.to_inner_uid(uid)])
    except ValueError: 
        return 0
    
def get_Ui(iid):
    try: 
        return len(trainset.ir[trainset.to_inner_iid(iid)])
    except ValueError:
        return 0
    
df_ = pd.DataFrame(predictions, columns=['uid', 'iid', 'rui', 'est', 'details'])
df_['Iu'] = df_.uid.apply(get_Iu)
df_['Ui'] = df_.iid.apply(get_Ui)
df_['err'] = abs(df_.est - df_.rui)
df_.sort_values(by='uid', ascending=True)
df_.head()

,uid,iid,rui,est,details,Iu,Ui,err
0,_hTADMIAWVZW91Fu0ebjcg,Heirloom Market BBQ,5.0,1.0,{'was_impossible': False},2,20,4.0
1,__48dJJcPvNgqUlEozwtpw,The Vortex Bar And Grill - Midtown,4.0,1.0,{'was_impossible': False},2,14,3.0
2,_02XN3yATdWwfMIbsGhMuQ,Big Dave's Cheesesteaks,4.0,1.0,{'was_impossible': False},3,6,3.0
3,0bywS0p66QN9dBt97EaErQ,Bacchanalia,4.0,1.0,{'was_impossible': False},11,9,3.0
4,03Jr73TBI998BHt6aiz9Dg,Canoe,4.0,1.0,{'was_impossible': False},4,17,3.0


In [93]:
#creating function to get top 5 Product Recommendation for each user.

def get_top_n(predictions, n=5):
    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [94]:
top_n = get_top_n(predictions, n=3)

In [95]:
# Print the recommended items for first 50 user

count=0
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])
    if(count>49):
        break
    count=count+1

_hTADMIAWVZW91Fu0ebjcg ['Antico Pizza', 'Heirloom Market BBQ']
__48dJJcPvNgqUlEozwtpw ['The Vortex Bar And Grill - Midtown', 'Cypress Street Pint & Plate']
_02XN3yATdWwfMIbsGhMuQ ["Big Dave's Cheesesteaks", '26 Thai Kitchen & Bar']
0bywS0p66QN9dBt97EaErQ ['Bacchanalia', 'Urban Wu', 'La Grotta Ristorante Italiano']
03Jr73TBI998BHt6aiz9Dg ['Canoe']
_McTiMJcKYdyXoo9110QrA ['La Tavola Trattoria']
_rvxykxj_yGsB_Mt9ugAUw ['Antico Pizza', 'Amalfi Pizza', 'Seven Lamps']
09Aqif6oShic-SDKGxF2Bg ['Gunshow']
_jLsE-7D1OleAx0JzfbB3A ['Marcel', 'Canoe']
_sIHo_g6xrTHxQCdiPmhGA ['Staplehouse', "Gus's World Famous Fried Chicken"]
0EwzsYbxuaGZSRKUQDNrGA ['Top Spice', 'Desta Ethiopian Kitchen']
_VLnawFgFOjmnQvZLNIKUg ['Old Lady Gang', 'Little Bangkok']
_hc9lla5PHSO4zK-90VvPw ["Poor Calvin's"]
_Z_bthCNWY2bMnaIAp7SYA ['Heirloom Market BBQ']
0Fm2MGb9E-4oxvrInm6NTw ['Barcelona Inman Park']
_C01nb941V68m-L-ta1WdA ['Two Urban Licks', 'Yard House - Takeout & Delivery Available']
_4Pcnyo17toXP1eE67PaJQ ['Farm Bur

In [311]:
import json

# writing
json.dump(top_n, open("rate_df_1.csv", 'w'))